本次作业以垃圾邮件分类任务为基础，要求提取文本特征并使用朴素贝叶斯算法进行垃圾邮件识别（调用已有工具包或自行实现）。

### 任务介绍
电子邮件是互联网的一项重要服务，在大家的学习、工作和生活中会广泛使用。但是大家的邮箱常常被各种各样的垃圾邮件填充了。有统计显示，每天互联网上产生的垃圾邮件有几百亿近千亿的量级。因此，对电子邮件服务提供商来说，垃圾邮件过滤是一项重要功能。而朴素贝叶斯算法在垃圾邮件识别任务上一直表现非常好，至今仍然有很多系统在使用朴素贝叶斯算法作为基本的垃圾邮件识别算法。

本次实验数据集来自[Trec06](https://plg.uwaterloo.ca/cgi-bin/cgiwrap/gvcormac/foo06)的中文垃圾邮件数据集，目录解压后包含三个文件夹，其中data目录下是所有的邮件（未分词），已分词好的邮件在data_cut目录下。邮件分为邮件头部分和正文部分，两部分之间一般有空行隔开。标签数据在label文件夹下，文件中每行是标签和对应的邮件路径。‘spam’表示垃圾邮件，‘ham’表示正常邮件。

本次实验

基本要求：
1. 提取正文部分的文本特征；
2. 划分训练集和测试集（可以借助工具包。一般笔记本就足够运行所有数据，认为实现困难或算力不够的同学可以采样一部分数据进行实验。）；
3. 使用朴素贝叶斯算法完成垃圾邮件的分类与预测，要求测试集准确率Accuracy、精准率Precision、召回率Recall均高于0.9（本次实验可以使用已有的一些工具包完成如sklearn）；
4. 对比特征数目（词表大小）对模型效果的影响；
5. 提交代码和实验报告。

扩展要求：
1. 邮件头信息有时也可以协助判断垃圾邮件，欢迎学有余力的同学们尝试；
2. 尝试自行实现朴素贝叶斯算法细节；
3. 尝试对比不同的概率计算方法。

### 导入工具包

In [16]:
'''
提示：
若调用已有工具包，sklearn中提供了一些可能会用到的类。
'''
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # 提取文本特征向量的类
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, ComplementNB # 三种朴素贝叶斯算法，差别在于估计p(x|y)的方式
from sklearn.model_selection import train_test_split, GridSearchCV #划分训练集
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score #评价指标

from pathlib import Path
from collections import Counter
RANDOM_SEED=2020

### 读取数据
读取数据，先读取index，获得标签和数据的路径
1. 将index文件变成 `std::map<std::filesystem::path, int>` file_label
2. 根据 _file\_label_ 将数据读取进来 ， 输出
    - std::vector\<int\> total_label_list
    - std::vector\<std::string\> total_header_list
    - std::vector\<std::string\> total_content_list

In [17]:
# hard code, for load label and datas
def GetDataPathandLabel(index_path:Path):
    # return std::map<std::filesystem::path , int>
    #   key = (label == "spam")? 1 : 0
    #   value = mail path
    file_label = {}
    with index_path.open('r', encoding="utf-8") as f:
        for each in f:
            line = each.split()   #  type(line) is list[str]
            if len(line) != 2:
                print(f"{each} -> invalid)")
                continue
            # garbage email is 1 , else 0
            label = 1 if line[0] == "spam" else 0

            relative_path = line[1].replace("data" , "data_cut")
            file_path = ( index_path.parent/relative_path).resolve()
            if( not file_path.exists()):
                continue
            file_label[file_path] = label
    return file_label


cwd = Path.cwd()
index_file_path = cwd/"trec06c-utf8/label/index"
file_label = GetDataPathandLabel(index_file_path)
    


In [18]:
# log for debug
debug_brief_count = Counter( file_label.values())
for label , count in debug_brief_count.items():
    print(f"label: {label} - count: {count}")

label: 1 - count: 42854
label: 0 - count: 21766


In [19]:
def SeperateLableHeaderContent(file_label):
    # input : std::map<std::filesystem::path , int> 
    # output: 
    #   std::tuple< 
    #       std::vector<int>,      // total_label_list
    #       std::vector<std::string>, // total_header_list
    #       std::vector<std::string>, // total_content_list
    #   >

    total_label_list=[]
    total_header_list=[]
    total_content_list=[]
    for each_path , label in file_label.items():
        with each_path.open('r' , encoding="utf-8") as f:
            each_mail_str = f.read()
        header = each_mail_str.split('\n\n',maxsplit=1)[0]
        content = each_mail_str.split('\n\n',maxsplit=1)[1]

        total_label_list.append(label)
        total_header_list.append(header)
        total_content_list.append(content)
    return total_label_list, total_header_list, total_content_list

total_label_list, total_header_list, total_content_list = SeperateLableHeaderContent(file_label)
print(f" Total label size {len(total_label_list)} with header/content size {len(total_header_list)}/{len(total_content_list)}  ")
print(total_content_list[0])

 Total label size 64620 with header/content size 64620/64620  
[ 课   程   背   景 ]

　
每 一位 管理 和 技术人员 都 清楚 地 懂得 ， 单纯 从 技术 角度 衡量 为 合算 的 方案 ， 也许
却是 一个 财务 陷阱 ， 表面 赢利 而 暗地里 亏损 ， 使经
营者 无法 接受 。 如何 将 技术手段 与 财务 运作 相结合 ， 使 每位 管理 和 技术人员 都 从
本 课程 通过 沙盘 模拟 和 案例 分析 ， 使 企业 各级 管理 和 技术人员 掌握 财务管理 知识
， 利用 财务 信息 改进 管理决策 ， 实现 管理 效益 最大化 。 通过 学习 本 课程 ， 您 将 ：
★   对 会计 与 财务管理 有 基本 了解 , 提高 日常 管理 活动 的 财务 可行性 ;
★   通过 分析 关键 业绩 指标 ， 形成 战略规划 与 全面 预算 ；
★   突出 企业 管理 的 重心 ， 形成 管理 的 系统性 。

[ 课   程   大   纲 ]

　

一 、 财务 工作 内容 及 作用
2 、 财务 专家 的 思维 模式
3 、 财务 工作 的 基本 内容
4 、 管理者 如何 利用 财务 进行 管理 和 决策
二 、 如何 阅读 和 分析 财务报表
1 、 会计报表 的 构成
2 、 损益表 的 阅读 与 分析
3 、 资产 负债表 的 阅读 与 分析
4 、 资金 流量 和 现金流量 表 的 阅读 与 分析
5 、 会计报表 之间 的 关系
◆ 案例 分析 ： 读 报表 ， 判断 企业 业绩 水平
三 、 如何 运用 财务 手段 进行 成本 控制
1 、 产品成本 的 概念 和 构成
2 、 Ｃ Ｖ Ｐ （ 本 Ａ 浚利 ） 分析 与 运用
3 、 标准 成本 制度 在 成本 控制 中 的 作用
4 、 如何 运用 目标 成本法 控制 产品成本 ， 保证 利润 水平
5 、 如何 运用 ABC 作业 成本法 进行 管理 分析 ， 实施 精细 成本 管理
6 、 如何 针对 沉没 成本 和 机会成本 进行 正确 决策
7 、 如何 改善 采购 、 生产 等 环节 的 运作 以 改良 企业 的 整体 财务状况
◆ 综合 案例 分析
1 、 管理 和 技术 方案 的 可行

### 将 文本 转换为 数值特征
  GPT: 词袋模型（Bag of Words) 是一种将文本转换为固定长度向量的方式，它将一段文本表示为一个词语出现频率的集合（袋子），忽略词序、语法和上下文，只关注每个词在文本中出现的次数（或权重）。

In [20]:
max_features = 6000
tfidf_vectorizer = TfidfVectorizer(
    token_pattern=r'(?u)\b\w\w+\b',      # (?u)：支持 unicode（匹配中文字符等）\b\w\w+\b：匹配边界内的，长度 ≥ 2 的单词（避免标点、单字符）
    max_features= max_features,                  # 最多提取1000个词（特征维度）
    min_df= 6   ,                         # 至少在6个文档中出现
    max_df= 0.667                        # 在66.7%以下的文档中出现
    )
tfidf_content = tfidf_vectorizer.fit_transform(total_content_list)
print(tfidf_vectorizer.vocabulary_)

{'一位': 845, '管理': 4794, '技术人员': 3424, '清楚': 4218, '懂得': 3306, '单纯': 2005, '技术': 3423, '角度': 5225, '衡量': 5164, '方案': 3704, '也许': 1230, '却是': 2035, '一个': 836, '财务': 5399, '表面': 5178, '赢利': 5448, '亏损': 1266, '无法': 3721, '接受': 3528, '如何': 2649, '运作': 5545, '相结合': 4564, '课程': 5367, '通过': 5652, '模拟': 4022, '案例': 4003, '分析': 1820, '企业': 1432, '各级': 2192, '掌握': 3520, '财务管理': 5403, '知识': 4612, '利用': 1856, '信息': 1579, '改进': 3612, '实现': 2758, '效益': 3631, '学习': 2689, '会计': 1465, '基本': 2466, '了解': 1243, '提高': 3564, '日常': 3736, '活动': 4151, '可行性': 2169, '关键': 1715, '业绩': 1096, '指标': 3499, '形成': 3144, '战略规划': 3352, '全面': 1675, '预算': 5897, '突出': 4726, '系统性': 4833, '工作': 2914, '内容': 1736, '作用': 1524, '专家': 1074, '思维': 3221, '模式': 4021, '管理者': 4807, '进行': 5600, '决策': 1768, '阅读': 5785, '财务报表': 5401, '会计报表': 1467, '构成': 3960, '资产': 5431, '资金': 5443, '流量': 4162, '之间': 1222, '关系': 1714, '报表': 3451, '判断': 1853, '水平': 4095, '运用': 5548, '手段': 3382, '成本': 3318, '控制': 3538, '概念': 4016, '标准': 3978, '制度': 1871, '目标

In [21]:
MNB = MultinomialNB()
BNB = BernoulliNB()
CNB = ComplementNB()

X_train, X_test, y_train, y_test = train_test_split(tfidf_content, total_label_list, 
                                                        test_size=0.2,
                                                        random_state=RANDOM_SEED)
MNB.fit(X_train , y_train)
y_predict = MNB.predict(X_test)
accuracy_score = accuracy_score(y_test,y_predict)
precision_score = precision_score(y_test,y_predict)
recall_score = recall_score(y_test,y_predict)
print(f"max_features: {max_features}, acc: {accuracy_score}, precision: {precision_score}, recall: {recall_score}")

max_features: 6000, acc: 0.9640204271123491, precision: 0.980278008791731, recall: 0.9650292397660819



对比记录：
    max_features: 5000, acc: 0.9630145465800062, precision: 0.9791073124406457, recall: 0.9646783625730995
    max_features: 6000, acc: 0.9640204271123491, precision: 0.980278008791731, recall:  0.9650292397660819


结论：
    当 max features大，各项指标都增高。特征向量的维度高，表明数据的利用率高，模型可以学到的内容就越多。
    但从推测角度触发，肯定不是越大越好，因为过大可能会过拟合。